# 1.问题分析

## 1.1.问题类别

## 1.2.特征分析 

## 1.3.数据挖掘

# 2.建立基线

## 2.1.准备工具包

In [2]:
import numpy as np
import pandas as pd
 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import xgboost as xgb
from lightgbm import LGBMRegressor
import math


from sklearn.model_selection import ShuffleSplit

from sklearn.model_selection import StratifiedKFold


%matplotlib inline

C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## 2.2.原始数据准备(不进行任何清洗)

In [3]:
train_raw = pd.read_csv('../data/public_raw.train.csv')
test_raw = pd.read_csv('../data/public_raw.test.csv')

train_raw['is_train']=1
test_raw['is_train']=0

df = pd.concat([train_raw, test_raw],sort=False)

rep_cols = {'ID':'ID', 
 '板温':'board_t', 
 '现场温度':'env_t', 
 '光照强度':'light_strength', 
 '转换效率':'efficiency', 
 '转换效率A':'efficiency_A', 
 '转换效率B':'efficiency_B', 
 '转换效率C':'efficiency_C', 
 '电压A':'V_A',
 '电压B':'V_B', 
 '电压C':'V_C', 
 '电流A':'I_A', 
 '电流B':'I_B', 
 '电流C':'I_C', 
 '功率A':'P_A', 
 '功率B':'P_B', 
 '功率C':'P_C', 
 '平均功率':'P_avg', 
 '风速':'wind_speed',
 '风向':'wind_direction', 
 '发电量':'y'
}

df.rename(index=str, columns=rep_cols, inplace=True)

df.sort_values(by=['ID'],ascending=True, inplace=True)

df.reset_index(drop=True, inplace=True)

## 2.3.特征工程(完全使用原始特征)

## 2.4.训练

### 2.4.1.准备工作

In [3]:
# CV时不方便shuffle，因此准备训练集和测试集时shuffle
df = df.sample(frac=1).reset_index(drop=True)

# 准备训练集和测试集
train = df[df['is_train']==1]
test = df[df['is_train']==0]
print('训练集数量:'+str(len(train)))
print('测试集数量:'+str(len(test)))

# 准备训练集合输入矩阵和输出矩阵
train_X = train.drop(['y','is_train'],axis=1)
train_Y = train['y']

# 准备测试集合输入矩阵
test_X = test.drop(['y','is_train'],axis=1)

# 准备测试集合输出矩阵容器
ans=pd.DataFrame()
ans['ID']=test_X['ID']


训练集数量:9000
测试集数量:8409


In [3]:
# 定义打分函数,  SCORE = 1/(1+RMSE)
def cal_score(mse):
    if isinstance(mse, float):
        return 1 / (1 + math.sqrt(mse))
    else:
        return np.divide(1, 1 + np.sqrt(mse))

# def cal_score(mse):
#     return np.divide(1, 1 + np.sqrt(mse))

# 定义交叉验证函数  
def cross_validate(models, X, Y, cv=5):
    model_name, mse_avg, score_avg = [], [], []
    for i, model in enumerate(models):
        #获取模型名
        name = str(i + 1) + '.' + str(model) 
#         print(i + 1,'- Model:', str(model).split('(')[0])
        print(name)
#         model_name.append(str(i + 1) + '.' + str(model).split('(')[0])
        model_name.append(name.split('(')[0])
        #计算metric
#         strat_k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2018)
#         folds=strat_k_fold.split(X,Y)
        #apply shuffling to cross_val_score
#         strat_k_fold = StratifiedKFold(n_splits=cv, shuffle=True, random_state=0)
        nmse = cross_val_score(model, X, Y, cv=cv, scoring='neg_mean_squared_error')
    
#         nmse = cross_val_score(model, X, Y, cv=cv, scoring='neg_mean_squared_error')
        avg_mse = np.average(-nmse)
        mse_avg.append(avg_mse)
        #计算分数
        scores = cal_score(-nmse)
        avg_score = np.average(scores)    
        score_avg.append(avg_score)
        print('MSE:', -nmse)
        print('Score:', scores)
        print('Average MSE:', avg_mse, ' - Score:', avg_score, '\n')
    res = pd.DataFrame()
    res['Model'] = model_name
    res['Avg MSE'] = mse_avg
    res['Avg Score'] = score_avg
    return res

### 2.4.2.Cross Validation（LightGBM/XGBoost/RF/GBM）

In [4]:
#基学习器

xgbt1 = xgb.XGBRegressor(n_estimators=950, max_depth=3, max_features='sqrt', random_state=321, n_jobs=8)
xgbt2 = xgb.XGBRegressor(n_estimators=1000, max_depth=3, max_features='sqrt', random_state=456, n_jobs=8)
xgbt3 = xgb.XGBRegressor(n_estimators=1100, max_depth=3, max_features='sqrt', random_state=789, n_jobs=8)
# n_estimators=1000  max_depth=5  'sqrt'  GradientBoostingRegressor 最佳参数 ,learning_rate=0.08
gbdt1 = GradientBoostingRegressor(n_estimators=800, max_depth=4, max_features='log2', random_state=123,learning_rate=0.08)
gbdt2 = GradientBoostingRegressor(n_estimators=900, max_depth=4, max_features='log2', random_state=456,learning_rate=0.08)
gbdt3 = GradientBoostingRegressor(n_estimators=1000, max_depth=5, max_features='log2', random_state=789,learning_rate=0.08)
# n_estimators=700, max_features='auto', random_state=2, n_jobs=8,max_depth=10
forest1 = RandomForestRegressor(n_estimators=800, max_features='sqrt', random_state=7, n_jobs=8)
forest2 = RandomForestRegressor(n_estimators=900, max_features='log2', random_state=9, n_jobs=8)
forest3 = RandomForestRegressor(n_estimators=900, max_features='sqrt', random_state=11, n_jobs=8) 

lgb1 = LGBMRegressor(n_estimators=900, max_depth=5, random_state=5, n_jobs=8) 
lgb2 = LGBMRegressor(n_estimators=850, max_depth=4, random_state=7, n_jobs=8)
lgb3 = LGBMRegressor(n_estimators=720, max_depth=4, random_state=9, n_jobs=8)


In [5]:
regrs = [
    xgbt1, gbdt1, forest1, lgb1,
    xgbt2, gbdt2, forest2, lgb2,
    xgbt3, gbdt3, forest3, lgb3
]

regrs_light = [
    lgb3, xgbt3, gbdt3, forest3
]

In [17]:
cross_validate(models=regrs_light, X = train_X, Y = train_Y)

1.LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=4, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=720,
       n_jobs=8, num_leaves=31, objective=None, random_state=9,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)
MSE: [0.01779459 0.02274581 0.01631306 0.06987453 0.0616137 ]
Score: [0.88230389 0.86894777 0.88674289 0.7909278  0.80114014]
Average MSE: 0.03766833784406778  - Score: 0.8460124974576955 

2.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, max_features='sqrt', min_child_weight=1, missing=None,
       n_estimators=1100, n_jobs=8, nthread=None, objective='reg:linear',
       random_state=789, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)
MSE: [0.0163

,Model,Avg MSE,Avg Score
0,1.LGBMRegressor,0.037668,0.846012
1,2.XGBRegressor,0.034380,0.855208
2,3.GradientBoostingRegressor,0.031493,0.856170
3,4.RandomForestRegressor,0.042011,0.836138


### 2.4.3.使用LightGBM/XGBoost/RF/GBM多种模型训练后融合

In [7]:
class Stacker(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models
    
    # Train_X: 原始训练集输入矩阵, Train_Y: 原始训练集输出矩阵, Test_X: 原始测试集输入矩阵
    def fit_predict(self, Train_X, Train_Y, Test_X):
        Train_X = np.array(Train_X)
        Train_Y = np.array(Train_Y)
        Test_X = np.array(Test_X)

        folds = list(KFold(n_splits=self.n_splits, shuffle=True, random_state=2018).split(Train_X, Train_Y))       
        
        # 以基学习器预测结果为特征的 stacker训练数据 与 stacker预测数据
        # 原始训练集预测结果容器
        S_train = np.zeros((Train_X.shape[0], len(self.base_models)))
        # 原始测试集预测结果容器
        S_predict = np.zeros((Test_X.shape[0], len(self.base_models)))
        
        for n_model, regr in enumerate(self.base_models):
            print(n_model + 1, 'Base model:', str(regr).split('(')[0])
            S_predict_i = np.zeros((Test_X.shape[0], self.n_splits))
            
            for n_fold, (train_idx, test_idx) in enumerate(folds):
                # 将X分为训练集与测试集
                X_train_fold, Y_train_fold, X_test_fold, Y_test_fold = Train_X[train_idx], Train_Y[train_idx], Train_X[test_idx], Train_Y[test_idx]
                print ('Fit fold', (n_fold+1), '...')
                regr.fit(X_train_fold, Y_train_fold)
                Y_pred = regr.predict(X_test_fold)
                # 每折训练得到的模型根据原始训练集中的测试折的输入矩阵预测
                S_train[test_idx, n_model] = Y_pred
                # 每折训练得到的模型根据原始测试集输入矩阵预测
                S_predict_i[:, n_fold] = regr.predict(Test_X)
            
            S_predict[:, n_model] = S_predict_i.mean(axis=1)

        nmse_score = cross_val_score(self.stacker, S_train, Train_Y, cv=5, scoring='neg_mean_squared_error')
        print('CV MSE:', -nmse_score)
        print('Stacker AVG MSE:', -nmse_score.mean(), 'Stacker AVG Score:', np.mean(np.divide(1, 1 + np.sqrt(-nmse_score))))

        self.stacker.fit(S_train, Train_Y)
        res = self.stacker.predict(S_predict)
        return res, S_train, S_predict

In [31]:
stacking_model = SVR(C=100, gamma=0.01, epsilon=0.01)
stacker = Stacker(5, stacking_model, regrs_light)
pred_stack, S_train_data, S_predict_data = stacker.fit_predict(train_X, train_Y, test_X)

1 Base model: XGBRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
2 Base model: GradientBoostingRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
3 Base model: RandomForestRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
4 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
CV MSE: [0.01488159 0.01912186 0.07429619 0.01533122 0.01309632]
Stacker AVG MSE: 0.027345434603574963 Stacker AVG Score: 0.8685469979970518


In [ ]:
#local CV: 0.7909689115097466

ans['y_n_shuffle'] = pred_stack
ans['y_n_shuffle'].describe()

In [115]:
#local CV: 0.8697213790417854

ans['y_shuffle'] = pred_stack
ans['y_shuffle'].describe()

count    8409.000000
mean        5.695812
std         3.459761
min        -0.131349
25%         2.505766
50%         5.704603
75%         8.887224
max        12.348426
Name: y_shuffle, dtype: float64

# 3.增加数据清洗工序

## 3.1.去重

In [20]:
# CV时不方便shuffle，因此准备训练集和测试集时shuffle
df = df.sample(frac=1).reset_index(drop=True)

# 准备训练集和测试集
train = df[df['is_train']==1]
test = df[df['is_train']==0]
print('训练集数量:'+str(len(train)))
print('测试集数量:'+str(len(test)))

# 对训练集进行去重
train.drop_duplicates(train.columns.drop('ID'), keep='first', inplace=True)
print('去重后训练集数量:'+str(len(train)))

# 准备训练集合输入矩阵和输出矩阵
train_X = train.drop(['y','is_train','I_B','I_C'],axis=1)
train_Y = train['y']

# 准备测试集合输入矩阵
test_X = test.drop(['y','is_train','I_B','I_C'],axis=1)

# 准备测试集合输出矩阵容器
ans=pd.DataFrame()
ans['ID']=test_X['ID']

训练集数量:9000
测试集数量:8409
去重后训练集数量:8918


C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [24]:
cross_validate(models=regrs_light, X = train_X, Y = train_Y)

1.LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=4, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=720,
       n_jobs=8, num_leaves=31, objective=None, random_state=9,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)
MSE: [0.01248851 0.05849184 0.01669115 0.03821168 0.03093328]
Score: [0.89948119 0.80524967 0.88558722 0.83648543 0.85042794]
Average MSE: 0.03136329128162621  - Score: 0.8554462916949553 

2.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, max_features='sqrt', min_child_weight=1, missing=None,
       n_estimators=1100, n_jobs=8, nthread=None, objective='reg:linear',
       random_state=789, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)
MSE: [0.0164

,Model,Avg MSE,Avg Score
0,1.LGBMRegressor,0.031363,0.855446
1,2.XGBRegressor,0.033967,0.849265
2,3.GradientBoostingRegressor,0.028054,0.863466
3,4.RandomForestRegressor,0.033721,0.850866


In [11]:
stacking_model = SVR(C=100, gamma=0.01, epsilon=0.01)
stacker = Stacker(5, stacking_model, regrs_light)
pred_stack, S_train_data, S_predict_data = stacker.fit_predict(train_X, train_Y, test_X)

1 Base model: XGBRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
2 Base model: GradientBoostingRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
3 Base model: RandomForestRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
4 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
CV MSE: [0.05783203 0.01160472 0.03428923 0.01559797 0.02176324]
Stacker AVG MSE: 0.028217437959844903 Stacker AVG Score: 0.8626126000810512


In [11]:
# 12 models stack, shuffle, local CV: 0.8710764504503057

ans['y_drop_duplicate'] = pred_stack
ans['y_drop_duplicate'].describe()

count    8409.000000
mean        5.696612
std         3.457480
min        -0.117571
25%         2.510122
50%         5.704925
75%         8.886383
max        12.132167
Name: y_drop_duplicate, dtype: float64

In [ ]:
# 4 models stack, shuffle train data, local CV: 0.8626126000810512

## 3.2.异常值处理:增加异常修正值特征(异常通过离群程度定义)

In [8]:
rolling_mask_two = [-i for i in range(1,2)]+[i for i in range(1,2)]
rolling_mask_four = [-i for i in range(1,3)]+[i for i in range(1,3)]
rolling_mask_six = [-i for i in range(1,4)]+[i for i in range(1,4)]
rolling_mask_eight = [-i for i in range(1,5)]+[i for i in range(1,5)]
rolling_mask_ten = [-i for i in range(1,6)]+[i for i in range(1,6)]

In [9]:
# 由于训练集中存在重复数据:1)连续重复数据 2)电学测量值为0的重复数据

# 对于训练集,重复项全部去除
train_raw = pd.read_csv('../data/public_raw.train.csv')
print('去重前训练集条数:' +str(train_raw.shape[0]))
# train_raw.drop_duplicates(train_raw.columns.drop('ID'), keep='first', inplace=True)
# print('去重后训练集条数:' +str(train_raw.shape[0]))

# 对于测试集,仅仅通过ID去除电学测量值为0的重复数据
test_raw = pd.read_csv('../data/public_raw.test.csv')

# 在去重前准备完整测试集ID
ans=pd.DataFrame()
ans['ID']=test_raw['ID']

# 电学测量量全为0的异常点个数
# zero_sample_ID = test_raw[test_raw[(test_raw == 0) | (test_raw == 0.)].count(axis=1) > 13]['ID'].tolist()
# print('测试集中电学测量量全为0的异常点个数为: '+str(len(zero_sample_ID)))

# 根据ID全部去除
print('去重前测试集条数:' +str(test_raw.shape[0]))
# test_raw = test_raw[~test_raw['ID'].isin(zero_sample_ID)].reset_index(drop=True)
# print('去重后测试集条数:' +str(test_raw.shape[0]))




train_raw['is_train']=1
test_raw['is_train']=0

df = pd.concat([train_raw, test_raw],sort=False)

rep_cols = {'ID':'ID', 
 '板温':'board_t', 
 '现场温度':'env_t', 
 '光照强度':'light_strength', 
 '转换效率':'efficiency', 
 '转换效率A':'efficiency_A', 
 '转换效率B':'efficiency_B', 
 '转换效率C':'efficiency_C', 
 '电压A':'V_A',
 '电压B':'V_B', 
 '电压C':'V_C', 
 '电流A':'I_A', 
 '电流B':'I_B', 
 '电流C':'I_C', 
 '功率A':'P_A', 
 '功率B':'P_B', 
 '功率C':'P_C', 
 '平均功率':'P_avg', 
 '风速':'wind_speed',
 '风向':'wind_direction', 
 '发电量':'y'
}

df.rename(index=str, columns=rep_cols, inplace=True)

df.sort_values(by=['ID'],ascending=True, inplace=True)

df.reset_index(drop=True, inplace=True)









去重前训练集条数:9000
去重前测试集条数:8409


In [38]:
# #计算偏差率的辅助列
# # for c in ['I_A','I_B','I_C','V_A','V_B','V_C']:
# for c in ['I_A','I_B','I_C']:
#     df[c+'_avg_sequence'] = np.nanmean([df[c].shift(i) for i in rolling_mask_eight],axis=0)
#     df[c+'_exception_ratio'] = np.abs(df[c]-df[c+'_avg_sequence'])/df[c+'_avg_sequence']
#     df[c+'_cor'] = df[c]
    
#     #out of range
#     oor_index = df[df[c]>20].index
#     print(oor_index)
    
#     outlier_index = df[df[c+'_exception_ratio']>1.6].index
#     print(outlier_index)
    
#     ab_index = pd.Int64Index(set(list(oor_index)+list(outlier_index)))
#     print(ab_index)
    
#     ab_data = df.loc[ab_index].sort_values(by='ID', ascending=True)
    
#     # 上下记录均值替代异常值
#     for idx, line in ab_data.iterrows():
#         ID = line['ID']
#         value = line[c]
        
#         index = df[df['ID'] == ID].index
            
#         before_offset = 1
#         while (idx + before_offset)in ab_index:
#             before_offset += 1

#         after_offset = 1
#         while (idx + after_offset) in ab_index:
#             after_offset += 1
    
#         print('ID :' + str(ID))
#         print(value, 'is abnormal as value of ',c)
#         replace_value = (df.loc[index - before_offset, c].values + df.loc[index + after_offset, c].values) / 2
#         df.loc[index, c+'_cor'] = replace_value[0]
#         print('Has been replaced by '+str(replace_value[0]))

        
        
        
        
        
# for c in ['V_A','V_B','V_C']:
#     df[c+'_avg_sequence'] = np.nanmean([df[c].shift(i) for i in rolling_mask_eight],axis=0)
#     df[c+'_exception_ratio'] = np.abs(df[c]-df[c+'_avg_sequence'])/df[c+'_avg_sequence']
#     df[c+'_cor'] = df[c]
    
#     #out of range
#     oor_index = df[(df[c]>800)|(df[c]<500)].index
#     print(oor_index)
    
#     outlier_index = df[df[c+'_exception_ratio']>1.6].index
#     print(outlier_index)
    
#     ab_index = pd.Int64Index(set(list(oor_index)+list(outlier_index)))
#     print(ab_index)
    
#     ab_data = df.loc[ab_index].sort_values(by='ID', ascending=True)
    
#     # 上下记录均值替代异常值
#     for idx, line in ab_data.iterrows():
#         ID = line['ID']
#         value = line[c]
        
#         index = df[df['ID'] == ID].index
            
#         before_offset = 1
#         while (idx + before_offset)in ab_index:
#             before_offset += 1

#         after_offset = 1
#         while (idx + after_offset) in ab_index:
#             after_offset += 1
    
#         print('ID :' + str(ID))
#         print(value, 'is abnormal as value of ',c)
#         replace_value = (df.loc[index - before_offset, c].values + df.loc[index + after_offset, c].values) / 2
#         df.loc[index, c+'_cor'] = replace_value[0]
#         print('Has been replaced by '+str(replace_value[0]))

        
        

        
# df['P_A_cor']=df['I_A_cor']*df['V_A_cor']
# df['P_B_cor']=df['I_B_cor']*df['V_B_cor']
# df['P_C_cor']=df['I_C_cor']*df['V_C_cor']
# df['P_avg_cor']=1/3*(df['P_A_cor']+df['P_B_cor']+df['P_C_cor'])


# # df.drop(columns=['I_A','I_B','I_C','V_A','V_B','V_C','P_A','P_B','P_C','P_avg'],axis=1,inplace=True)
# # df.drop(columns=['I_A_avg_sequence','I_A_exception_ratio','I_B_avg_sequence','I_B_exception_ratio','I_C_avg_sequence','I_C_exception_ratio','V_B_avg_sequence','V_B_exception_ratio','V_A_avg_sequence','V_A_exception_ratio','V_C_avg_sequence','V_C_exception_ratio',],axis=1,inplace=True)
# # 0.85661



            

Int64Index([], dtype='int64')
Int64Index([  527,   529,   531,   673,   859,  1373,  1561,  1722,  2071,
             3108,  3253,  3255,  3303,  3305,  3311,  3316,  3318,  3429,
             4671,  4672,  5083, 16413],
           dtype='int64')
Int64Index([  527,   529,   531,  2071,  1561, 16413,   673,  3108,  3253,
             3255,  1722,  4671,  4672,  5083,   859,  1373,  3429,  3303,
             3305,  3311,  3316,  3318],
           dtype='int64')
ID :591.0
6.68 is abnormal as value of  I_A
Has been replaced by 0.9400000000000001
ID :593.0
6.68 is abnormal as value of  I_A
Has been replaced by 1.56
ID :595.0
6.77 is abnormal as value of  I_A
Has been replaced by 0.9650000000000001
ID :737.0
6.53 is abnormal as value of  I_A
Has been replaced by 0.345
ID :948.0
6.87 is abnormal as value of  I_A
Has been replaced by 0.33
ID :1519.0
7.04 is abnormal as value of  I_A
Has been replaced by 0.77
ID :1717.0
6.81 is abnormal as value of  I_A
Has been replaced by 0.33999999999999997


KeyError: "None of [Int64Index([-1], dtype='int64')] are in the [index]"

In [9]:
#原始路线
all_data = df.copy()
bad_feature = ['ID','P_A', 'P_B', 'P_C', 'P_avg', 'env_t', 'V_A', 'V_B', 'V_C', 'I_B', 'I_C', 'efficiency', 'efficiency_A', 'efficiency_B', 'efficiency_C']
bad_index1 = all_data[bad_feature][
    (all_data[bad_feature] > all_data[bad_feature].mean() + 2 * all_data[bad_feature].std()) | 
    (all_data[bad_feature] < all_data[bad_feature].mean() - 2 * all_data[bad_feature].std())
].dropna(how='all').index
bad_index2 = all_data[
    ((all_data['V_A']<500)&(all_data['V_A']!=0))|
    ((all_data['V_B']<500)&(all_data['V_B']!=0))|
    ((all_data['V_C']<500)&(all_data['V_C']!=0))].index
bad_index = pd.Int64Index(list(bad_index1)+list(bad_index2))

# bad_index = all_data[bad_feature][
#     (all_data[bad_feature] > all_data[bad_feature].mean() + 2 * all_data[bad_feature].std()) | 
#     (all_data[bad_feature] < all_data[bad_feature].mean() - 2 * all_data[bad_feature].std())
# ].dropna(how='all').index



bad_data = all_data.loc[bad_index].sort_values(by='ID', ascending=True)





# 上下记录均值替代异常值
for idx, line in bad_data.iterrows():
    ID = line['ID']
    col_index = line[bad_feature][ 
        (line[bad_feature] > all_data[bad_feature].mean() + 3 * all_data[bad_feature].std())| 
        (line[bad_feature] < all_data[bad_feature].mean() - 3 * all_data[bad_feature].std())
    ].index
    index = all_data[all_data['ID'] == ID].index
    
    # idx - before_offset, CV  0.8684
    before_offset = 1
    while (idx + before_offset)in bad_index:
        before_offset += 1

    after_offset = 1
    while (idx + after_offset) in bad_index:
        after_offset += 1
    
    replace_value = (all_data.loc[index - before_offset, col_index].values + all_data.loc[index + after_offset, col_index].values) / 2
    all_data.loc[index, col_index] = replace_value[0]

In [12]:
# CV时不方便shuffle，因此准备训练集和测试集时shuffle
# df = df.sample(frac=1).reset_index(drop=True)
df = all_data.copy()

df = df.sample(frac=1).reset_index(drop=True)

# 准备训练集和测试集
train = df[df['is_train']==1]
test = df[df['is_train']==0]
print('训练集数量:'+str(len(train)))
print('测试集数量:'+str(len(test)))

# 训练集中电学测量值为0的条目在构建完特征后失去价值，通过去重去掉，避免人为改变权重影响训练
train = train.drop_duplicates(train.columns.drop(['ID']), keep='first')
print('去重后训练集条数:' +str(train.shape[0]))


# 准备训练集合输入矩阵和输出矩阵
train_X = train.drop(['y','is_train'],axis=1)
train_Y = train['y']

# 准备测试集合输入矩阵
test_X = test.drop(['y','is_train'],axis=1)

# 准备测试集合输出矩阵容器
ans=pd.DataFrame()
ans['ID']=test_X['ID']

训练集数量:9000
测试集数量:8409
去重后训练集条数:8918


In [13]:
cross_validate(models=regrs_light, X = train_X, Y = train_Y)

1.LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=4, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=720,
       n_jobs=8, num_leaves=31, objective=None, random_state=9,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)
MSE: [0.01685016 0.01555956 0.01799888 0.04183777 0.05964575]
Score: [0.88510601 0.88909599 0.88170987 0.83019048 0.80371328]
Average MSE: 0.03037842546310161  - Score: 0.8579631268893145 

2.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, max_features='sqrt', min_child_weight=1, missing=None,
       n_estimators=1100, n_jobs=8, nthread=None, objective='reg:linear',
       random_state=789, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)
MSE: [0.0158

,Model,Avg MSE,Avg Score
0,1.LGBMRegressor,0.030378,0.857963
1,2.XGBRegressor,0.032686,0.852430
2,3.GradientBoostingRegressor,0.029740,0.861150
3,4.RandomForestRegressor,0.031433,0.856706


In [64]:
# 训练集未shuflle时
stacking_model = SVR(C=100, gamma=0.01, epsilon=0.01)
stacker = Stacker(5, stacking_model, regrs_light)
pred_stack, S_train_data, S_predict_data = stacker.fit_predict(train_X, train_Y, test_X)

1 Base model: XGBRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
2 Base model: GradientBoostingRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
3 Base model: RandomForestRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
4 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
CV MSE: [0.03820138 0.01411471 0.01500888 0.01062797 0.08564431]
Stacker AVG MSE: 0.03271945123206558 Stacker AVG Score: 0.860264287075261


In [10]:
stacking_model = SVR(C=100, gamma=0.01, epsilon=0.01)
stacker = Stacker(5, stacking_model, regrs_light)
pred_stack, S_train_data, S_predict_data = stacker.fit_predict(train_X, train_Y, test_X)

1 Base model: XGBRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
2 Base model: GradientBoostingRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
3 Base model: RandomForestRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
4 Base model: LGBMRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
CV MSE: [0.02937148 0.03176862 0.01071031 0.03045894 0.0620819 ]
Stacker AVG MSE: 0.032878251749318445 Stacker AVG Score: 0.8521156276701483


# 4.增加特征工程工序

## 4.1.增加前后有效发电量均值

In [9]:
#前二后二
next_one = []
prev_one = []
next_id = []
prev_id = []

second_next_one = []
second_prev_one = []

df_len = df.shape[0]

i_y =df.columns.get_loc("y")

def get_prev_nn_index(cur_i):
    prev_i = cur_i-1
    while(prev_i>=0 and pd.isnull(df.iat[prev_i,i_y])):
        prev_i-=1
    return prev_i

def get_next_nn_index(cur_i):
    prev_i = cur_i+1
    while(prev_i<df_len and pd.isnull(df.iat[prev_i,i_y])):
        prev_i+=1
    return prev_i

for i in range(df_len):
    f_pre_i=get_prev_nn_index(i)
    if(f_pre_i)<0:
        prev_one.append(np.nan)
        prev_id.append(0)
    else:
        prev_one.append(df.iat[f_pre_i,i_y])
        prev_id.append(f_pre_i)
        
    s_pre_i=get_prev_nn_index(f_pre_i)
    if (s_pre_i)<0:
        second_prev_one.append(np.nan)
    else:
        second_prev_one.append(df.iat[s_pre_i,i_y])
    
    f_next_i=get_next_nn_index(i)
    if(f_next_i<df_len):
        next_one.append(df.iat[f_next_i,i_y])
        next_id.append(f_next_i)
    else:
        next_one.append(np.nan)
        next_id.append(df_len)
    
    s_next_i=get_next_nn_index(f_next_i)
    if(s_next_i<df_len):
        second_next_one.append(df.iat[s_next_i,i_y])
    else:
        second_next_one.append(np.nan)
        

df['next_value'] = next_one
df['prev_value'] = prev_one
df['avg_value'] = np.nanmean([df['next_value'], df['prev_value']],axis=0)

df.drop(['next_value','prev_value'],1,inplace=True)

In [51]:
def add_avg(df):
    array = np.array(df["P_avg"])
    newarray=[]
    num = 0
    for i in np.arange(len(array)):
        for j in np.arange(10):
            if i<10:
                num = (array[j-1]+array[j-2]+array[j-3])/3
            if i>=10:
                num = (array[i-1]+array[i-2]+array[i-3]+array[i-5]+array[i-6]+array[i-7]+array[i-8]+array[i-9])/9
        newarray.append(num)
    df["old_SoCalledSF_P_avg"] = newarray
    return df

df = add_avg(df)

In [11]:
# CV时不方便shuffle，因此准备训练集和测试集时shuffle
# train_test_set
tt = df.copy()

tt = tt.sample(frac=1).reset_index(drop=True)

# 准备训练集和测试集
train = tt[tt['is_train']==1]
test = tt[tt['is_train']==0]
print('训练集数量:'+str(len(train)))
print('测试集数量:'+str(len(test)))

# 准备训练集合输入矩阵和输出矩阵
train_X = train.drop(['y','is_train'],axis=1)
train_Y = train['y']

# 准备测试集合输入矩阵
test_X = test.drop(['y','is_train'],axis=1)

# 准备测试集合输出矩阵容器
ans=pd.DataFrame()
ans['ID']=test_X['ID']


训练集数量:9000
测试集数量:8409


In [12]:
cross_validate(models=regrs_light, X = train_X, Y = train_Y)

1.LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=4, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=720,
       n_jobs=8, num_leaves=31, objective=None, random_state=9,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)
MSE: [0.01821326 0.01621795 0.08931631 0.01187811 0.02710777]
Score: [0.88109104 0.88703619 0.7699069  0.90172402 0.8586312 ]
Average MSE: 0.032546681038947124  - Score: 0.8596778691461715 

2.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, max_features='sqrt', min_child_weight=1, missing=None,
       n_estimators=1100, n_jobs=8, nthread=None, objective='reg:linear',
       random_state=789, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)
MSE: [0.018

,Model,Avg MSE,Avg Score
0,1.LGBMRegressor,0.032547,0.859678
1,2.XGBRegressor,0.031564,0.860170
2,3.GradientBoostingRegressor,0.032007,0.860731
3,4.RandomForestRegressor,0.037897,0.849747


In [10]:
#前二后二
next_one = []
prev_one = []
next_id = []
prev_id = []

second_next_one = []
second_prev_one = []

df_len = all_data.shape[0]

i_y =all_data.columns.get_loc("y")

def get_prev_nn_index(cur_i):
    prev_i = cur_i-1
    while(prev_i>=0 and pd.isnull(all_data.iat[prev_i,i_y])):
        prev_i-=1
    return prev_i

def get_next_nn_index(cur_i):
    prev_i = cur_i+1
    while(prev_i<df_len and pd.isnull(all_data.iat[prev_i,i_y])):
        prev_i+=1
    return prev_i

for i in range(df_len):
    f_pre_i=get_prev_nn_index(i)
    if(f_pre_i)<0:
        prev_one.append(np.nan)
        prev_id.append(0)
    else:
        prev_one.append(all_data.iat[f_pre_i,i_y])
        prev_id.append(f_pre_i)
        
    s_pre_i=get_prev_nn_index(f_pre_i)
    if (s_pre_i)<0:
        second_prev_one.append(np.nan)
    else:
        second_prev_one.append(all_data.iat[s_pre_i,i_y])
    
    f_next_i=get_next_nn_index(i)
    if(f_next_i<df_len):
        next_one.append(all_data.iat[f_next_i,i_y])
        next_id.append(f_next_i)
    else:
        next_one.append(np.nan)
        next_id.append(df_len)
    
    s_next_i=get_next_nn_index(f_next_i)
    if(s_next_i<df_len):
        second_next_one.append(all_data.iat[s_next_i,i_y])
    else:
        second_next_one.append(np.nan)
        

all_data['next_value'] = next_one
all_data['prev_value'] = prev_one
all_data['avg_value'] = np.nanmean([all_data['next_value'], all_data['prev_value']],axis=0)

all_data.drop(['next_value','prev_value'],1,inplace=True)

In [14]:
def add_avg(df):
    array = np.array(df["P_avg"])
    newarray=[]
    num = 0
    for i in np.arange(len(array)):
        for j in np.arange(10):
            if i<10:
                num = (array[j-1]+array[j-2]+array[j-3])/3
            if i>=10:
                num = (array[i-1]+array[i-2]+array[i-3]+array[i-5]+array[i-6]+array[i-7]+array[i-8]+array[i-9])/9
        newarray.append(num)
    df["old_SoCalledSF_P_avg"] = newarray
    return df

all_data = add_avg(all_data)

In [16]:
all_data.head()

,ID,board_t,env_t,light_strength,efficiency,efficiency_A,efficiency_B,efficiency_C,V_A,V_B,...,P_A,P_B,P_C,P_avg,wind_speed,wind_direction,y,is_train,avg_value,old_SoCalledSF_P_avg
0,1,0.01,0.1,1,0.00,0.00,0.00,0.00,0.0,0.0,...,0.00,0.00,0.00,0.00,0.1,1,NaN,0,1.437752,1172.806667
1,9,-19.33,-17.5,13,198.32,259.11,42.17,293.66,722.0,705.0,...,909.72,148.05,1031.03,696.27,0.3,273,NaN,0,1.437752,1172.806667
2,10,-19.14,-17.4,34,80.55,106.32,16.98,118.36,729.0,709.0,...,976.86,155.98,1087.50,740.11,0.6,272,1.437752,1,1.692575,1172.806667
3,11,-18.73,-17.3,30,99.90,139.00,21.20,139.51,728.0,717.0,...,1128.40,172.08,1132.56,811.01,0.8,275,1.692575,1,1.706770,1172.806667
4,12,-17.54,-17.0,41,82.48,114.86,14.91,117.66,731.0,722.0,...,1279.25,166.06,1310.40,918.57,1.1,283,1.975787,1,2.031615,1172.806667


In [17]:
# CV时不方便shuffle，因此准备训练集和测试集时shuffle
# train_test_set
tt = all_data.copy()

tt = tt.sample(frac=1).reset_index(drop=True)

# 准备训练集和测试集
train = tt[tt['is_train']==1]
test = tt[tt['is_train']==0]
print('训练集数量:'+str(len(train)))
print('测试集数量:'+str(len(test)))

train = train.drop_duplicates(train.columns.drop(['ID','avg_value','old_SoCalledSF_P_avg']), keep='first')
print('去重后训练集条数:' +str(train.shape[0]))

# 准备训练集合输入矩阵和输出矩阵
train_X = train.drop(['y','is_train'],axis=1)
train_Y = train['y']

# 准备测试集合输入矩阵
test_X = test.drop(['y','is_train'],axis=1)

# 准备测试集合输出矩阵容器
ans=pd.DataFrame()
ans['ID']=test_X['ID']

训练集数量:9000
测试集数量:8409
去重后训练集条数:8918


In [18]:
cross_validate(models=regrs_light, X = train_X, Y = train_Y)

1.LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=4, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=720,
       n_jobs=8, num_leaves=31, objective=None, random_state=9,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)
MSE: [0.01574406 0.01823915 0.01963925 0.07508936 0.01916174]
Score: [0.88851349 0.8810166  0.87708519 0.78491433 0.87840586]
Average MSE: 0.02957470992734625  - Score: 0.861987095604432 

2.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, max_features='sqrt', min_child_weight=1, missing=None,
       n_estimators=1100, n_jobs=8, nthread=None, objective='reg:linear',
       random_state=789, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)
MSE: [0.01804

,Model,Avg MSE,Avg Score
0,1.LGBMRegressor,0.029575,0.861987
1,2.XGBRegressor,0.030152,0.860076
2,3.GradientBoostingRegressor,0.026449,0.869302
3,4.RandomForestRegressor,0.029301,0.861345
